In [1]:
from sqlite3 import connect
import datetime as dt
import functools
from astropy.io import fits
from astropy.coordinates import SkyCoord, Angle
from astropy.table import Table, QTable, join
from astropy.time import Time
import astropy.units as u
import  astroplan as ap # import FixedTarget, Observer, AltitudeConstraint, AirmassConstraint, AtNightConstraint, is_observable
import pandas as pd


In [2]:
conn = connect("../django/QuadEBdb/db.sqlite3")

targets = pd.read_sql("select * from tom_target;", conn, index_col="id")

speckle = pd.read_sql("select * from tom_specklerawdata;", conn, index_col="target_id")
spectrum = pd.read_sql(
    "select * from tom_spectrumrawdata;", conn, index_col="target_id"
)

targets = targets.join(speckle.groupby("target_id").id.agg(num_speckle="count")).rename(columns={"num_speckle": "Num Speckle"})
targets = targets.join(spectrum.groupby("target_id").id.agg(num_spectra="count")).rename(columns={"num_spectra": "Num Spectra"})
targets.fillna(0, inplace=True)
targets['Num Speckle'] = targets['Num Speckle'].astype(int)
targets['Num Spectra'] = targets['Num Spectra'].astype(int)

for target_list_tup in conn.execute("select name from tom_targetlist;").fetchall():
    target_list = target_list_tup[0]
    list_members = [result[0] for result in conn.execute("""
                                select t.local_id
                                from tom_targetlist tl
                                join tom_targetlist_targets tlt on tlt.targetlist_id = tl.id
                                join tom_target t on t.id = tlt.target_id
                                where tl.name = ?
                                ;
                                """, [target_list]).fetchall()]
    targets[target_list] = False
    targets.loc[targets.local_id.isin(list_members), target_list] = True

targets

,local_id,source,ra,dec,pmra,pmdec,distance,magnitude,Num Speckle,Num Spectra,Featured targets,HQND,Kostov 2022,Kostov 2023,LBT 2023-12-06,Speckle Resolved
id,,,,,,,,,,,,,,,,
2574,TIC 270360534,Kostov 2023 arXiv:2309.14200,263.430502,43.860752,-11.961609,-23.987384,573.281395,11.813276,0,0,False,False,False,True,False,False
2575,TIC 219469945,Kostov 2022 arXiv:2202.05790,241.047908,43.030301,-0.866116,-13.091347,923.125385,12.429089,24,0,False,False,True,False,False,False
2576,TIC 20212631,Kostov 2023 arXiv:2309.14200,227.038060,39.970240,-82.825344,60.712753,186.291719,10.515903,0,0,False,False,False,True,False,False
2577,TIC 150055835,Kostov 2023 arXiv:2309.14200,69.925310,29.134584,1.092154,-1.994131,2877.829357,14.611104,0,0,False,False,False,True,False,False
2578,TIC 161043618,Kostov 2022 arXiv:2202.05790,223.425148,52.715834,4.313893,-0.286929,923.184076,12.273352,18,2,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2767,TIC 414969157,Kostov 2022 arXiv:2202.05790,141.176162,22.200757,-0.138778,-0.350210,784.409022,14.067556,15,0,False,False,True,False,False,False
2768,TIC 27543409,Kostov 2022 arXiv:2202.05790,122.702004,13.567217,-3.266658,0.942613,1929.040762,13.569499,15,0,False,False,True,False,False,False
2769,TIC 139914081,Kostov 2023 arXiv:2309.14200,326.971651,-49.037832,-4.706880,-3.407268,823.054112,14.392851,0,0,False,False,False,True,False,False


In [3]:
fixed_targets = [
    ap.FixedTarget(
        coord=SkyCoord(
            frame="icrs",
            obstime=Time("2000.0", format="jyear", scale="tdb"),
            ra=target["ra"] * u.deg,
            dec=target["dec"] * u.deg,
            pm_ra_cosdec=target["pmra"] * u.mas / u.yr,
            pm_dec=target["pmdec"] * u.mas / u.yr,
            distance=target["distance"] * u.pc,
        ),
        name=target["local_id"],
    )
    for (_, target) in targets.iterrows()
]

observer = ap.Observer.at_site("lbt")

constraints = [
    ap.AltitudeConstraint(30*u.deg, 80*u.deg),
    ap.AirmassConstraint(2),
    # ap.AtNightConstraint.twilight_civil(),
    ]

observing_nights_utc = [
    '2024-01-13',
    '2024-01-14',
    '2024-01-15',
    '2024-01-16',
    '2024-01-17',
    '2024-01-18',
]

targets["Observable Nights"] = 0
for observing_night_utc in observing_nights_utc:
    night = Time(observing_night_utc)
    observing_session = [observer.twilight_evening_civil(night, which='nearest'), observer.twilight_morning_civil(night, which='next')]
    observable_tonight = ap.is_observable(constraints, observer, fixed_targets, observing_session)
    targets.loc[observable_tonight, "Observable Nights"] += 1


In [4]:
criteria={
    # "test": [(targets["HQND"]), ""],
    "Other Bright with spectra and speckle": [(targets["magnitude"] < 11) & (targets["Num Speckle"] > 0) & (targets["Num Spectra"] > 0) & (targets["HQND"] == False), "Low"],
    "HQND, Dim": [(targets["HQND"]) & (targets["magnitude"]>13), "Low"],
    "HQND Bright no spectra": [(targets["HQND"]) & (targets["Num Spectra"] == 0) & (targets["magnitude"] < 13), "Medium"],
    "HQND Bright with spectra": [(targets["HQND"]) & (targets["Num Spectra"] > 0) & (targets["magnitude"] < 13), "High"],
    "Featured": [(targets["Featured targets"]), "Highest"],
}

matching_ids = {}
combined_ids = set()
targets["Priority"] = ""
for name, [criterion, priority] in criteria.items():
    filtered_ids = set(targets[criterion  & (targets["Observable Nights"] > 0)]["local_id"])
    targets.loc[targets.local_id.isin(filtered_ids), "Priority"]=priority
    print(f"{len(filtered_ids):3d} targets from criterion: {name}")
    combined_ids=combined_ids | filtered_ids
    matching_ids[name] = filtered_ids
print(f"Total of {len(combined_ids)} targets")

print()
for name,ids in matching_ids.items():
    print(f"Criteria {name}:")
    print(f"    {ids}")

  5 targets from criterion: Other Bright with spectra and speckle
  1 targets from criterion: HQND, Dim
  3 targets from criterion: HQND Bright no spectra
  6 targets from criterion: HQND Bright with spectra
  2 targets from criterion: Featured
Total of 15 targets

Criteria Other Bright with spectra and speckle:
    {'TIC 307119043', 'TIC 389836747', 'TIC 367448265', 'TIC 322727163', 'TIC 470710327'}
Criteria HQND, Dim:
    {'TIC 459959916'}
Criteria HQND Bright no spectra:
    {'TIC 264402353', 'TIC 284814380', 'TIC 63459761'}
Criteria HQND Bright with spectra:
    {'TIC 414026507', 'TIC 391620600', 'TIC 454140642', 'TIC 123098844', 'TIC 278352276', 'TIC 52856877'}
Criteria Featured:
    {'TIC 470710327', 'TIC 307119043'}


In [5]:
#data from Robbie's LBT observing readme, with dupes removed manually
import numpy as np
mag=np.array([7.9,9.4,9.5,10,10.2,10.3,10.6,10.7,10.8,10.9,11,11.1,11.2,11.3,11.4,11.5,11.7,11.8,11.9,12,12.1,12.3,12.4,12.5,12.6,12.7,12.8,13.2,13.3,13.4,13.5,13.6,13.7,13.8,13.9,])
sec=np.array([60,60,60,60,60,60,60,60,60,72,72,84,78,84,102,102,132,138,150,192,180,240,270,258,270,318,360,510,594,630,666,726,768,858,918,])

from scipy.interpolate import CubicSpline
cs = CubicSpline(mag, sec)

print("TargetName       RA (J2000)     DEC (J2000)       Gmag     pmRA     pmDEC     Mode               ExposureTime   ExecutionTime        Priority  Notes")
print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
for _,target in targets[targets["local_id"].isin(combined_ids)].sort_values("ra").iterrows():
    exposure = cs(target["magnitude"])
    readout_sec = exposure + 140
    readout_min = readout_sec / 60
    print(
        f"{target["local_id"]:<15s} "
        f"{Angle(target["ra"]*u.deg).to_string(unit=u.hour,decimal=False,precision=2,sep=":"):>11s}    "
        f"{Angle(target["dec"]*u.deg).to_string(unit=u.deg,decimal=False,precision=2,sep=":",alwayssign=True):>12s}      "
        f"{target["magnitude"]:5.2f}    "
        f"{target["pmra"]:5.2f}    "
        f"{target["pmdec"]:6.2f}     "
        f"F300/CDII/CDVI     "
        f"1x{exposure:4.0f}sec     "
        f"{readout_sec:5.0f} sec = {readout_min:4.1f}min   "
        f"{target["Priority"]:<7s}   "
        "--"
    )

TargetName       RA (J2000)     DEC (J2000)       Gmag     pmRA     pmDEC     Mode               ExposureTime   ExecutionTime        Priority  Notes
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
TIC 284814380    0:47:27.28    +64:49:04.86      12.02    -2.56     -0.24     F300/CDII/CDVI     1x 193sec       333 sec =  5.6min   Medium    --
TIC 307119043    0:59:18.61    +51:13:17.91       9.98     7.56     -4.11     F300/CDII/CDVI     1x  60sec       200 sec =  3.3min   Highest   --
TIC 52856877     1:09:20.23    +61:02:28.48      10.85     3.22     -1.72     F300/CDII/CDVI     1x  66sec       206 sec =  3.4min   High      --
TIC 389836747    1:33:10.91    +61:35:07.14      10.71    -3.38     -4.47     F300/CDII/CDVI     1x  60sec       200 sec =  3.3min   Low       --
TIC 454140642    4:19:05.63     +0:54:00.15      10.22     2.83    -10.85     F30

In [6]:
print("#TargetName      RA(J2000)      DEC(J2000)        VMag      MODE               t_exp")
for _,target in targets[targets["local_id"].isin(combined_ids)].sort_values("ra").iterrows():
    exposure = cs(target["magnitude"])
    readout_sec = exposure + 140
    readout_min = readout_sec / 60
    print(
        f"{target["local_id"]:<15s}  "
        f"{Angle(target["ra"]*u.deg).to_string(unit=u.hour,decimal=False,precision=2,sep=":"):>11s}    "
        f"{Angle(target["dec"]*u.deg).to_string(unit=u.deg,decimal=False,precision=2,sep=":",alwayssign=True):>12s}      "
        f"{target["magnitude"]:5.2f}     "
        f"F300/CDII/CDVI     "
        f"1x{exposure:4.0f}sec     "
    )

#TargetName      RA(J2000)      DEC(J2000)        VMag      MODE               t_exp
TIC 284814380     0:47:27.28    +64:49:04.86      12.02     F300/CDII/CDVI     1x 193sec     
TIC 307119043     0:59:18.61    +51:13:17.91       9.98     F300/CDII/CDVI     1x  60sec     
TIC 52856877      1:09:20.23    +61:02:28.48      10.85     F300/CDII/CDVI     1x  66sec     
TIC 389836747     1:33:10.91    +61:35:07.14      10.71     F300/CDII/CDVI     1x  60sec     
TIC 454140642     4:19:05.63     +0:54:00.15      10.22     F300/CDII/CDVI     1x  60sec     
TIC 459959916     4:45:57.52     +4:49:46.63      13.23     F300/CDII/CDVI     1x 536sec     
TIC 391620600     4:46:33.69    +44:45:13.18      12.49     F300/CDII/CDVI     1x 259sec     
TIC 367448265     5:13:31.79    +35:39:10.99       7.87     F300/CDII/CDVI     1x  60sec     
TIC 123098844    18:38:17.48    +44:41:54.96      11.11     F300/CDII/CDVI     1x  84sec     
TIC 278352276    20:30:00.87    +48:36:25.40      10.32     F300/CDII